In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [4]:
import nvidia_smi

nvidia_smi.nvmlInit()
deviceCount = nvidia_smi.nvmlDeviceGetCount()
freem = []
for i in range(deviceCount):
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    freem.append(100*info.free/info.total)
nvidia_smi.nvmlShutdown()
print(freem)

[98.6397777234978, 98.68812082722684]


# 1. 데이터 로드

In [5]:
from langchain.document_loaders import CSVLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [6]:
# 사용 문서명 입력
docs_nm = './rag_data/rerere_answer_summary_by_chatgpt.csv'
train_loader = CSVLoader(docs_nm, 'contents', ['idx'])
documents = train_loader.load()

&#10004; 블로그 컨텐츠 추가

In [ ]:
docs_nm = './rag_data/blog_summary_final.csv'
blog_loader = CSVLoader(docs_nm)
blog_documents = blog_loader.load()

documents = documents + blog_documents

# 2. RAG

In [9]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from peft import LoraConfig, PeftModel

## 2.1 모델 로드

In [10]:
model_nm = "Orion-14B-Chat-RAG-papering"

In [11]:
base_model = "jylee420/Orion-14B-Chat-RAG-papering"

In [12]:
# 4-bit quantization configuration
compute_dtype = getattr(torch, "float16")
use_4bit=True

quant_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [13]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [14]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
print(tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token, tokenizer.unk_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id)

# tokenizer.pad_token = tokenizer.eos_token # <eos> 토큰 안나오는 문제 원인일 수 있음
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

None None <unk> <unk>
None None 0 0


In [15]:
%%time
model = AutoModelForCausalLM.from_pretrained(
                base_model,
                quantization_config=quant_config,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True,
)

Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


CPU times: user 50.6 s, sys: 13.7 s, total: 1min 4s
Wall time: 12.1 s


In [16]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=3000,
                device_map="auto",
                torch_dtype=torch.float16
               )

In [17]:
llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={'temperature':0.1, 'max_length':3000}
)

## 2.2 Retriever

In [18]:
#load Embedding model
# embedding_model = "jhgan/ko-sbert-nli"
embedding_model = "BAAI/bge-m3" # 정연전임님 임베딩 모델
model_kwargs={'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [19]:
%%time
vectordb = FAISS.from_documents(
    documents = documents,
    embedding = embeddings,
)

CPU times: user 25min 18s, sys: 6min 43s, total: 32min 1s
Wall time: 2min 3s


#### multi query retriever

In [20]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import LLMChain

In [21]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

&#10004; 다중질문 처리하는 prompttemplate 추가. inference할 때 prompt 추가하는 것 대신 아래 항목에서 제어하면 됨

In [22]:
QUERY_PROMPT = PromptTemplate(
    input_variables=['question'],
    template = """
        Your task is to split multiple querys to single query that aim to answer all user's questions 
        The user questions are focused on interior, papering, and related techniques.
        Each query Must tackle the question from a different viewpoint, we want to get a variety of RELEVANT search results.
        Provide these alternative questions seperated by newlines.
        Original question : {question}
    """
)

In [23]:
# 선택할 관련 문서 개수 정하기
# search_type = "similarity_score" 가 기본버전
retriever = vectordb.as_retriever(search_type="mmr", # "similarity_score
                                  search_kwargs={"k":3, 'fetch_k':3})

&#10004; MultiqueryRetriever을 한 번 더 선언함

In [24]:
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = retriever,
    llm = llm,
    prompt = QUERY_PROMPT,
)

&#10004; chain 안의 retriever를 retriever_from_llm으로 변경해주어야 함

In [25]:
# Chaining a pipeline
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever_from_llm,
    chain_type='stuff',
    # combine_docs_chain_kwargs={'prompt': QUERY_PROMPT},
)

# 3. Evaluation

In [27]:
def rag_answer(query) :
    chat_history = []
    result = chain({"question": query, "chat_history": chat_history})
    return result['answer']

&#10004; rag_answer 함수를 실행시키면 사용자 질문이 어떤 다양한 쿼리로 변형되는지 확인 가능함

In [28]:
question = "유성페인트를 사용하는 것에 대한 부작용이 있을까요? 또한, 페인트가 남으면 어디에 보관하는 게 좋을까요?"

In [29]:
rag_answer(question)

/usr/local/lib/python3.8/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=1024) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 유성페인트 사용시 부작용은 무엇인가요?', '        2. 유성페인트를 사용하지 않는다면 어떤 대안이 있을까요?', '        3. 유성페인트를 사용한 후 남은 페인트는 어떻게 처리해야 할까요?', '        4. 유성페인트를 사용하기 전 주의사항은 무엇인가요?', '        5. 유성페인트를 사용한 후 냄새가 오래가는 경우가 있는데, 왜 그런가요?']
Both `max_new_tokens` (=1024) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_

' 유성페인트를 사용할 때 발생할 수 있는 부작용으로는 인체에 유해한 영향, 강한 냄새, 건조 시간이 길어지는 것 등이 있습니다. 또한, 환경 오염, 유지보수의 어려움, 비용 증가 등이 있습니다. 따라서, 유성페인트를 사용할 때는 안전하고 적절한 환경에서 사용하고, 꼭 필요한 경우에만 사용하는 것이 좋습니다. 페인트가 남았을 때는 깨끗한 컨테이너에 보관하고, 날짜와 종류를 표시하는 것이 좋습니다.'

In [31]:
test = pd.read_csv('./dacon_result/test.csv')
test.head(2)

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?


In [32]:
%%time
result = []
docs_lst = []
for q in tqdm(test['질문']) :
    result.append(rag_answer(q))
    docs_lst.append(retriever_from_llm.get_relevant_documents(query=q))

  0%|                                                                                                                                                           | 0/130 [00:00<?, ?it/s]Both `max_new_tokens` (=1024) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 방청 페인트의 종류와 특징은 무엇인가요?', '        2. 원목사이딩을 사용하면 어떤 문제가 발생할 수 있나요?', '        3. 방청 페인트를 선택할 때 고려해야 할 사항은 무엇인가요?', '        4. 원목사이딩의 장단점은 무엇인가요?']
Both `max_new_tokens` (=1024) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=1024) and `max_length`(=3000) seem to have been set. `max_new_tokens` will tak

CPU times: user 34min 59s, sys: 40.2 s, total: 35min 39s
Wall time: 21min 37s


## 3.2 인코딩

In [33]:
distinct = "retriever_multiquery_blog"
today_dt = "2024-03-09"
test_nm = f"{today_dt}_{model_nm}_{distinct}.csv"

In [34]:
import numpy as np
from sentence_transformers import SentenceTransformer
# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [35]:
pred_embeddings = model.encode(result)
pred_embeddings.shape

(130, 512)

In [36]:
submit = pd.read_csv('./dacon_result/sample_submission.csv')
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.056146,0.012225,0.048413,-0.000903,0.065287,-0.023057,-0.037039,0.019635,0.056421,...,-0.018746,-0.032642,-0.010175,-0.031516,-0.025673,0.007156,0.044942,0.024686,0.062100,0.009575
1,TEST_001,0.040271,0.039443,-0.009193,-0.004355,0.088474,-0.043821,0.004259,0.000764,-0.028205,...,-0.016111,-0.032665,0.019138,-0.032185,-0.018601,0.052290,-0.007165,-0.008496,0.026450,0.035253
2,TEST_002,0.027215,-0.041707,-0.036222,-0.002769,0.125714,-0.042054,-0.002592,-0.028100,0.046394,...,-0.054712,-0.008952,0.020674,-0.026505,0.011675,0.021572,-0.040035,-0.016765,-0.036097,0.067451
3,TEST_003,-0.045773,0.021728,-0.000468,0.031362,-0.042158,-0.061704,-0.006765,-0.062369,0.055340,...,0.006900,-0.019480,0.060088,-0.027113,0.006861,0.023011,-0.014874,0.026047,-0.030009,0.088995
4,TEST_004,-0.030955,-0.004856,0.025146,-0.025958,0.111109,0.010313,0.062251,0.005962,-0.022942,...,-0.006018,0.043498,0.037011,0.046339,-0.010173,-0.021279,0.012555,0.055604,0.021586,0.088097


In [38]:
submit.to_csv('./dacon_result/for_submission/submission_'+test_nm, index=False)

## 3.2 답변 결과 저장

In [39]:
%%time
docs_lst2 = []
for d in docs_lst :
    doc = []
    for x in d :
        doc.append(x.metadata)
    docs_lst2.append(doc)

CPU times: user 148 µs, sys: 18 µs, total: 166 µs
Wall time: 168 µs


In [40]:
test['result'] = result
test['rag_docs'] = docs_lst2
test

,id,질문,result,rag_docs
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용...","방청 페인트는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트...",[{'source': '질문 : 방청 페인트는 어떤 기능을 가지고 있나요? 방청 ...
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?,"도배지에 녹이 묻어나는 주된 이유는 방청제 사용 누락, 제품 불량, 방화문 프레임...",[{'source': './rag_data/blog_summary_final.csv...
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?",큐블럭의 단점은 가격이 비싸고 균열이 쉽게 생기며 습기로 인해 하자가 발생할 수 ...,[{'source': '질문 : 큐블럭은 어떤 건물 구조물을 만들 때 사용되는 건축...
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있...",고층 건물의 외단열 공사 방법에는 비계 설치와 이동식 비계 등이 있습니다.,[{'source': '질문 : 고층건물의 단열 시공은 어떻게 해야 하나요? 답변 ...
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?,도배 후 도배지가 완전히 건조되기까지 최대 일주일이 걸릴 수 있습니다. 그러나 일...,[{'source': '질문 : 도배 후 건조는 얼마나 해야 해? 답변 : 도배 ...
...,...,...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...,"분말 소화기를 사용할 때에는 압력계의 눈금 위치를 확인하고, 받침대를 사용하여 부...",[{'source': '질문 : 분말 소화기가 뭐야? 분말 소화기는 어떤 용도로 ...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?""",압출법 보온판의 가장 큰 장점은 압축 강도가 높고 시공이 간편하며 내부식성과 내수...,"[{'source': '질문 : 압출법 보온판은 어떤 재료로 만들어지고, 어떤 용도..."
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...,"수성 벽체용 탄성 방수 도료는 우수한 방수 기능을 제공하며, 크랙의 방지와 물세척...",[{'source': '질문 : 수성 방수 도료를 사용하는 것이 왜 중요할까요? ...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...,"석고보드 부분 교체(미장보수)의 원인은 도배 시공 후 석고보드를 부분교체하거나, ...",[{'source': './rag_data/blog_summary_final.csv...


In [41]:
test['result'].iloc[0]

' 방청 페인트는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등으로 분류됩니다. 원목사이딩을 사용하는 단점은 가격대가 높고 관리가 어렵고 습기에 민감하여 변형 가능성이 있으며, 색상 변색과 해충 침입에 취약하다는 것입니다.'

In [42]:
test.to_csv('./dacon_result/'+test_nm, encoding='utf-8-sig', index=False)